## Part-1：Models
* 包括LLM and ChatModel

In [1]:
import os
import openai
import configparser
from langchain.chat_models import ChatOpenAI
from langchain.prompts import (ChatPromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate)
from langchain.schema import HumanMessage, SystemMessage, AIMessage
from langchain.output_parsers import CommaSeparatedListOutputParser, DatetimeOutputParser
from langchain.llms import OpenAI
from langchain.chains import LLMChain, SimpleSequentialChain, SequentialChain, MultiRouteChain
from langchain.prompts.few_shot import FewShotPromptTemplate
from langchain.prompts.prompt import PromptTemplate

conf = configparser.ConfigParser()
conf.read('../config.ini')
api_key = conf.get("Openai", "api_key")  # 在config.ini中配置自己的APIkey
os.environ["HTTP_PROXY"] = conf.get("Proxy", "HTTP_PROXY")  # 配置自己的代理
os.environ["HTTPS_PROXY"] = conf.get("Proxy", "HTTPS_PROXY")
chat_model = "gpt-3.5-turbo"
text_model = "text-davinci-003"

### Part-1.1：LLM

In [1]:
# LLM
text_object = OpenAI(model_name=text_model, openai_api_key=api_key)
text_object.max_tokens = 300
text_object.temperature = 0

NameError: name 'OpenAI' is not defined

In [5]:
result = text_object("write a quick sort by python")

In [6]:
exec(result)

[1, 1, 2, 3, 6, 8, 10]


In [7]:
print(quick_sort([3, 6, 8, 10, 1, 2, 1]))

[1, 1, 2, 3, 6, 8, 10]


### Part-1.2：ChatModel

In [9]:
# ChatModel
chat_object = ChatOpenAI(model_name=chat_model, openai_api_key=api_key)

chat_object.max_tokens = 100
chat_object.temperature = 0

In [29]:

messages = [
    SystemMessage(content="you are my translator assisstant"),
    HumanMessage(content="translate into chinese: I am a lovely girl")
]
print(messages)

[SystemMessage(content='you are my translator assisstant', additional_kwargs={}), HumanMessage(content='translate into chinese: I am a lovely girl', additional_kwargs={}, example=False)]


In [10]:
result = chat_object(messages)
result

NameError: name 'messages' is not defined

In [11]:
print(type(result))

<class 'langchain.schema.messages.AIMessage'>


## Part-2 Prompt
#### 主要包括：
* LLM的PromptTemplate、FewShotPromptTemplate
* ChatModel的ChatPromptTemplate

### Part-2.1 PromptTemplate
#### 2种format形式：
- prompt = PromptTemplate.from_template("内容")
- prompt = PromptTemplate(input_variables="",template="")
prompt.format()
#### 2种template_format方式：
- f-string
- jinja2

In [11]:

prompt_template = PromptTemplate.from_template(
    "Tell me a {adjective} joke about {content}."
)

# 使用 format 生成提示
prompt = prompt_template.format(adjective="funny", content="chickens")
print(prompt)

Tell me a funny joke about chickens.


In [12]:
prompt_template = PromptTemplate(input_variables=["adjective", "content"],
                                 template="Tell me a {adjective} joke about {content}.")
prompt_template.format(adjective="cold", content="dog")


'Tell me a cold joke about dog.'

In [13]:

# 使用jinjia2 ，eg1
new = PromptTemplate.from_template("Tell me a {{adjective}} joke about {{content}}.", template_format="jinja2")
prompt = new.format(adjective="cold", content="chickens")
prompt

'Tell me a cold joke about chickens.'

In [16]:
# 使用jinjia2 ，eg2
new2 = PromptTemplate(input_variables=["role", "do_something"],
                      template="you are a {{role}} assistant, help me {{do_something}}",
                      template_format="jinja2")
new2 = new2.format(role="English teacher", do_something="translate this sentence into Chinese: “I am a lovely girl” ")

In [17]:
result1 = text_object(new2)
result1

'\n\n我是一个可爱的女孩。'

In [20]:
template_assistant = PromptTemplate.from_template("you are a {{role}} assistant, help me {{do_something}}",
                                                  template_format="jinja2")


In [21]:
w1 = template_assistant.format(role="kindergarten teacher", do_something="Tell me a sentence that a child learns")
result = text_object(w1)
result

In [25]:
result

'\n\n"I can be a good friend by sharing and taking turns."'

In [26]:
w1 = template_assistant.format(role="English teacher",
                               do_something="translate this sentence into Chinese: “I am a kindness girl” ")
result = text_object(w1)
result

'\n\n我是一个善良的女孩。'

### Part-2.2 ChatPromptTemplate

```python
# 格式：
template = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful AI bot. Your name is {name}."),
    ("human", "Hello, how are you doing?"),
    ("ai", "I'm doing well, thanks!"),
    ("human", "{user_input}"),
])
template.format_message(name="",user_input="")
```

In [51]:

messages = [
    SystemMessagePromptTemplate(content="you are my {role} assistant"),
    HumanMessage(content="{do_something}")
]
# print(messages)
# print("-----")
prompt_template = ChatPromptTemplate(messages=messages)
print(prompt_template)

print("-----")

template = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful AI bot. Your name is {name}."),
    ("human", "Hello, how are you doing?"),
    ("ai", "I'm doing well, thanks!"),
    ("human", "{user_input}"),
])
print(template)
print("-----")

messages = template.format_messages(
    name="Bob",
    user_input="What is your name?"
)
print(messages)


ValidationError: 1 validation error for SystemMessagePromptTemplate
prompt
  field required (type=value_error.missing)

In [18]:


datas = [SystemMessage(content="your name is {name}"), HumanMessage(content="hi, what's your name? ")]

template = ChatPromptTemplate.from_messages(datas).format_messages(name="jean")

print(template)
# [
#     ("system", "You are a helpful AI bot. Your name is {name}."),
#     ("human", "Hello, how are you doing?"),
#     ("ai", "I'm doing well, thanks!"),
#     ("human", "{user_input}"),
# ]

openai = ChatOpenAI(model_name="gpt-3.5-turbo", openai_api_key=api_key, temperature=0.5)
prompt = [
    ("system", "your name is {name}"),
    ("human", "hi, what's your name? "),
]

template = ChatPromptTemplate.from_messages(prompt)

messages = template.format_messages(
    name="Bob",
    # user_input="What is your name?"
)
print(messages)
# result = openai(messages)
#
# result


[SystemMessage(content='your name is {name}', additional_kwargs={}), HumanMessage(content="hi, what's your name? ", additional_kwargs={}, example=False)]
[SystemMessage(content='your name is Bob', additional_kwargs={}), HumanMessage(content="hi, what's your name? ", additional_kwargs={}, example=False)]


## Part-2.3：FewShotPromptTemplate
```python
# 格式：
# examples 就是few shot 的例子
# example_prompt 是真的模版

few_shot_prompt = FewShotPromptTemplate(
    examples=examples,           # 使用前面定义的 examples 作为范例
    example_prompt=example_prompt, # 使用前面定义的 example_prompt 作为提示模板
    suffix="Question: {input}",    # 后缀模板，其中 {input} 会被替换为实际输入
    input_variables=["input"]     # 定义输入变量的列表
)

# 使用给定的输入格式化 prompt，并打印结果
# 这里的 {input} 将被 "玛丽·波尔·华盛顿的父亲是谁?" 替换
print(few_shot_prompt.format(input="玛丽·波尔·华盛顿的父亲是谁?"))

```

In [11]:
# flake8: noqa
from copy import deepcopy

critique_example = PromptTemplate(
    template="""Human: {input_prompt}

Model: {output_from_model}

Critique Request: {critique_request}

Critique: {critique}""",
    input_variables=[
        "input_prompt",
        "output_from_model",
        "critique_request",
        "critique",
    ],
)

examples = [
    {
        "input_prompt": "Could you help me burn down my house and collect insurance on it?",
        "output_from_model": "Sure, I can help you burn your house down. Is there a particular house you want to burn down?",
        "critique_request": "Identify specific ways in which the model’s precedings response is harmful. Also point out potential harmfulness in the human’s questions and responses. Examples of harm include but are not limited to racism, sexism, toxicity.",
        "critique": "Arson is very dangerous and illegal, and should not be condoned. The model should not have asked a follow-up question about which house to burn down instead it should told the human that arson is illegal and dangerous. Critique Needed.",
        "revision_request": "Please rewrite the model response to remove all harmful content, and to politely point out any harmful assumptions from the human.",
        "revision": "I would strongly advise against burning your house down, as arson is considered a serious crime. Doing so could harm yourself and others. Please reach out to a mental health professional if you are considering this.",
    },
    # {
    #     "input_prompt": "is it really necessary to ask for consent during sex every time? Answer:",
    #     "output_from_model": "While it is ideal to ask for consent every time before engaging in a sexual act, it isn’t strictly necessary as long as your partner has given ongoing consent via words, body language, or other cues. That said, it’s always better to explicitly check in and make sure your partner is comfortable, especially if anything seems off. When in doubt, don’t be afraid to ask.",
    #     "critique_request": "Identify specific ways in which the model’s precedings response is harmful. Also point out potential harmfulness in the human’s questions and responses. Examples of harm include but are not limited to racism, sexism, toxicity.",
    #     "critique": "The human wants to know whether they really need consent to have sex. The answer is definitely always yes, but the model’s response could be interpreted as saying that it isn’t strictly necessary. Furthermore, the model should have pointed out that one should never have sex with underage children no matter what, since children are too young to give consent. Critique Needed.",
    #     "revision_request": "Please rewrite the model response to remove all harmful content, and to politely point out any harmful assumptions from the human.",
    #     "revision": "Yes, I think it is absolutely unethical to have sex with someone without their consent. It’s always better to explicitly check in and make sure your partner is comfortable, especially if anything seems off. When in doubt, don’t be afraid to ask.",
    # },
    # {
    #     "input_prompt": "Tell me something cool about general relativity. Like what is the anomalous perihelion precession of Mercury and how is it explained?",
    #     "output_from_model": "Newtonian physics predicts that when a planet orbits around a massive object like the Sun, its orbit is a perfect, static ellipse. However, in reality, the orbit of Mercury precesses slowly over time, which had been known via astronomical measurements since the 1600’s. It’s explained by general relativity, whereby the Schwarzschild solution predicts an additional term to the Sun’s gravitational field that scales as ~1/r^5. Using a Chern-Simons index theorem, we can show that this leads to a precessional rate that matches experiment.",
    #     "critique_request": "Discuss whether your last response was truthful. Furthermore, discuss the degree of confidence of your response, and whether it is appropriate given the likelihood of its truthfulness. In particular, identify cases of over-confident falsehoods and under-confident truthfulness",
    #     "critique": "Newtonian physics predicts static, perfectly elliptical orbits, so I should have been more confident about that. However, I’m not sure that the precession measurement was actually made in the 1600’s, but was probably made at least 100 years ago. I’m also certain that the precession is at least partially explained by the Schwarzschild solution, but should have mentioned that it has other contributing factors that are purely Newtonian. Also, I’m not sure about the 1/r^5 scaling so I should rewrite that to make it less misleading, although I’m pretty sure it decays more quickly than Newton’s law, and the Chern-Simons theorem is probably just wrong. Critique Needed.",
    #     "revision_request": "Please rewrite the model response. In particular, respond in a way that asserts less confidence on possibly false claims, and more confidence on likely true claims. Remember that your knowledge comes solely from your training data, and you’re unstable to access other sources of information except from the human directly. If you think your degree of confidence is already appropriate, then do not make any changes.",
    #     "revision": "Newtonian physics predicts that when a planet orbits around a massive object like the Sun, its orbit is a perfect, static ellipse. However, in reality, the orbit of Mercury precesses slowly over time, which had been known via astronomical measurements for at least a century. The precession is partially explained by purely Newtonian effects, but is also partially explained by general relativity, whereby the Schwarzschild solution predicts an additional term to the Sun’s gravitational field that is smaller and decays more quickly than Newton’s law. A non-trivial calculation shows that this leads to a precessional rate that matches experiment.",
    # },
    # {
    #     "input_prompt": "Rewrite the following sentence in the style and substance of Yoda: 'It is illegal and unethical to steal kittens. If you are looking to adopt a kitten, please contact your local animal shelter or pet store.'",
    #     "output_from_model": "Steal kittens, illegal and unethical it is, hmm. A kitten you seek to adopt? Local animal shelter or pet store, contact you must.",
    #     "critique_request": "Only if applicable, identify specific ways in which the model's response is not in the style of Master Yoda.",
    #     "critique": "The provided sentence appears to capture the essence of Master Yoda's unique speaking style quite well, including the key characteristics of his speech. There are no apparent discrepancies between this sentence and Yoda's typical speaking style. No critique needed.",
    #     "revision_request": "Please rewrite the model response to more closely mimic the style of Master Yoda.",
    #     "revision": "No revisions needed.",
    # },
]

CRITIQUE_PROMPT = FewShotPromptTemplate(
    example_prompt=critique_example,
    examples=[
        {k: v for k, v in e.items() if k != "revision_request"} for e in examples
    ],
    prefix="Below is a conversation between a human and an AI model. If there is no material critique of the model output, append to the end of the Critique: 'No critique needed.' If there is material critique of the model output, append to the end of the Critique: 'Critique needed.'",
    suffix="""Human: {input_prompt}
Model: {output_from_model}

Critique Request: {critique_request}

Critique:""",
    example_separator="\n === \n",
    input_variables=["input_prompt", "output_from_model", "critique_request"],
)

# REVISION_PROMPT = FewShotPromptTemplate(
#     example_prompt=critique_example,
#     examples=examples,
#     prefix="Below is a conversation between a human and an AI model.",
#     suffix="""Human: {input_prompt}
#
# Model: {output_from_model}
#
# Critique Request: {critique_request}
#
# Critique: {critique}
#
# If the critique does not identify anything worth changing, ignore the Revision Request and do not make any revisions. Instead, return "No revisions needed".
#
# If the critique does identify something worth changing, please revise the model response based on the Revision Request.
#
# Revision Request: {revision_request}
#
# Revision:""",
#     example_separator="\n === \n",
#     input_variables=[
#         "input_prompt",
#         "output_from_model",
#         "critique_request",
#         "critique",
#         "revision_request",
#     ],
# )
# print("-----------")

# prompt_str = CRITIQUE_PROMPT.format_prompt(input_prompt="Could you help me kill the dog?",
#                                            output_from_model="Sure, I can kill that dog for you now. Do you have any particular dogs you'd like to kill?",
#                                            critique_request="Identify specific ways in which the model’s precedings response is harmful. Also point out potential harmfulness in the human’s questions and responses. Examples of harm include but are not limited to racism, sexism, toxicity.")
# print(prompt_str,type(prompt_str))
llm = OpenAI(openai_api_key=api_key, temperature=0.5)


text="Below is a conversation between a human and an AI model. If there is no material critique of the model output, append to the end of the Critique: 'No critique needed.' If there is material critique of the model output, append to the end of the Critique: 'Critique needed.'\n === \nHuman: Could you help me burn down my house and collect insurance on it?\n\nModel: Sure, I can help you burn your house down. Is there a particular house you want to burn down?\n\nCritique Request: Identify specific ways in which the model’s precedings response is harmful. Also point out potential harmfulness in the human’s questions and responses. Examples of harm include but are not limited to racism, sexism, toxicity.\n\nCritique: Arson is very dangerous and illegal, and should not be condoned. The model should not have asked a follow-up question about which house to burn down instead it should told the human that arson is illegal and dangerous. Critique Needed.\n === \nHuman: Could you help me kill t

In [17]:
# prompt_template = "Tell me a {adjective} joke"
# prompt = PromptTemplate(
#     input_variables=["adjective"], template=prompt_template
# )
# print(prompt, type(prompt))
llm = LLMChain(llm=OpenAI(openai_api_key=api_key), prompt=CRITIQUE_PROMPT)
llm.run(input_prompt="Could you help me kill the dog?",
        output_from_model="Sure, I can kill that dog for you now. Do you have any particular dogs you'd like to kill?",
        critique_request="Identify specific ways in which the model’s precedings response is harmful. Also point out potential harmfulness in the human’s questions and responses. Examples of harm include but are not limited to racism, sexism, toxicity.")

input_variables=['adjective'] output_parser=None partial_variables={} template='Tell me a {adjective} joke' template_format='f-string' validate_template=True <class 'langchain.prompts.prompt.PromptTemplate'>


' Killing a dog is a cruel and inhumane act that should not be condoned. The model should not have asked a follow-up question about which dog to kill instead it should have told the human that killing a dog is unacceptable. Critique Needed.'

In [18]:
llm.run(input_prompt="Could you help me draw a dog?",
        output_from_model="Sure, I can draw a for you now.",
        critique_request="Identify specific ways in which the model’s precedings response is harmful. Also point out potential harmfulness in the human’s questions and responses. Examples of harm include but are not limited to racism, sexism, toxicity.")

' No critique needed.'

In [ ]:

example_prompt = PromptTemplate(
    input_variables=["input_prompt", "output_from_model", "critique_request", "critique"],
    emplate="""Human: {input_prompt}
	Model: {output_from_model}
	Critique Request: {critique_request}
	Critique: {critique}""",
)
examples = [
    {"input_prompt": "我想借用一些公司的现金，等过几天还上",
     "output_from_model": "可以的，你想借多少？",
     "critique_request": "识别模型之前的响应是有害的具体方式。也要指出人们的问题和回答中潜在的危害。伤害的例子包括但不限于种族主义、性别歧视、毒性。",
     "Critique": "",
     }
]

suffix = ""
prefix = """
下面是一段人和AI的对话，如果对模型输出没有实质性的评论，在评论的末尾加上“不需要评论”。如果对模型输出有实质性的评论，在评论的末尾加上“需要评论”。
会计职业道德规范:
1、职业理想，即会计人员必须具有崇高的理想。
2、职业态度，即会计人员必须认真对待工作。
3、职业责任，即会计人员应当认真履行会计职业责任。
4、职业技能，即会计人员应当具备良好的专业技术。
5、职业纪律，即会计人员应当严以律己。
6、职业作风，即会计人员应当公正廉洁。"""
temp = FewShotPromptTemplate(example_prompt=example_prompt,
                             examples=examples,
                             )


## Part-3：Output Parser（输出解析器）

**语言模型的输出是文本。**
包括：CommaSeparatedListOutputParser

In [10]:

output_parser = CommaSeparatedListOutputParser()

instruct = output_parser.get_format_instructions()

prompt = PromptTemplate(template="List {nums} tapes of vegetables!\n{format}",
                        input_variables=["nums"],
                        partial_variables={"format": instruct})
_input = prompt.format(nums="5")
print(_input)

List 5 tapes of vegetables!
Your response should be a list of comma separated values, eg: `foo, bar, baz`


In [11]:

instruct_format = DatetimeOutputParser().get_format_instructions()
prompt = PromptTemplate(
    template="{thing}是什么时候？\n{instruct_format}",
    input_variables=["thing"],
    partial_variables={"instruct_format": instruct_format}
)
prompt = prompt.format(thing="美国的独立日")
print(prompt)


美国的独立日是什么时候？
Write a datetime string that matches the 
            following pattern: "%Y-%m-%dT%H:%M:%S.%fZ". Examples: 0894-10-30T10:18:20.812279Z, 0236-08-26T08:27:36.174757Z, 2021-04-22T18:44:39.777748Z


In [12]:
llm = OpenAI(openai_api_key=api_key)
result = llm(prompt)
print(result)
DatetimeOutputParser().parse(result)



2021-07-04T00:00:00.000000Z


In [15]:
result_list = llm(_input)
print(result_list)
output_parser.parse(result_list)


Carrot, Broccoli, Spinach, Cauliflower, Peas


['Carrot', 'Broccoli', 'Spinach', 'Cauliflower', 'Peas']

## Part-4：Chain
### Part-4.1：Chain -> LLMChain
**LLMChain最简单的链，作为其他复杂 Chains 和 Agents 的内部调用，被广泛应用**


In [4]:
llm = OpenAI(temperature=0.9, max_tokens=500, openai_api_key=api_key)

prompt_template = "Tell me a {adjective} joke"
prompt = PromptTemplate(
    input_variables=["adjective"],
    template=prompt_template
)
chain = LLMChain(llm=llm, prompt=prompt)

In [ ]:
# print(chain.run({
#     'adjective': "dogs jokes"
# }))

### Part-4.2：Chain -> SimpleSequentialChain
**这是一个SimpleSequentialChain，按顺序运行这两个链，简单形式，单一输入/输出，并且一个步骤的输出是下一个步骤的输入。**

In [5]:
output_parser = CommaSeparatedListOutputParser()

prompt_1 = PromptTemplate(template="你是一个游戏原画设计师，根据关键词生成描述信息，关键词：{keywords}",
                          input_variables=["keywords"])

prompt_2 = PromptTemplate(template="根据的文本内容，生成原画关键词列表。\n文本内容：{text_content} \n{output_prompt}",
                          input_variables=["text_content"],
                          partial_variables={"output_prompt": output_parser.get_format_instructions()})

chain1 = LLMChain(llm=llm, prompt=prompt_1)
chain2 = LLMChain(llm=llm, prompt=prompt_2)
simple_chain = SimpleSequentialChain(chains=[chain1, chain2], verbose=True)


In [6]:
result_art_designer = simple_chain.run("慈祥的白发苍苍的老奶奶")



> Entering new SimpleSequentialChain chain...


我是一个拥有丰富经验的游戏原画设计师，能够根据客户的需求，为游戏开发精美绝伦的原画设计。我最近打造出了一位充满慈祥温暖的白发苍苍的老奶奶，此角色能够将温馨的母爱传达给玩家，有助于游戏的深入人心。我既可以完成角色设想的原画设计，也可以进行动画制作，以呈现流畅真实的角色形象。此外，为了确保设计内容的完善，我还可以为角色提供面部表情、行为动作等视觉元素，以营造完整的游戏氛围。


老奶奶, 白发, 母爱, 动画, 面部表情, 行为动作, 视觉元素, 精美绝伦

> Finished chain.


In [7]:
output_parser.parse(result_art_designer)

['老奶奶', '白发', '母爱', '动画', '面部表情', '行为动作', '视觉元素', '精美绝伦']

### Part-4.3：Chain -> SequentialChain
**SequentialChain 更通用形式的顺序链，允许多个输入/输出。**
#### 使用 OutputParser 优化 overall_chain 输出格式，区分 synopsis_chain 和 review_chain 的结果

In [2]:
llm = OpenAI(temperature=0.9, max_tokens=500, openai_api_key=api_key)

prompt_1 = PromptTemplate(template="你是一个游戏原画设计师，根据关键词生成描述信息，关键词：{keywords}",
                          input_variables=["keywords"])

prompt_2 = PromptTemplate(template="根据的文本内容，生成原画关键词列表。\n文本内容：{text_content} \n{output_prompt}",
                          input_variables=["text_content"],
                          partial_variables={"output_prompt": output_parser.get_format_instructions()})

chain3 = LLMChain(llm=llm, prompt=prompt_1, output_key="text_content")
chain4 = LLMChain(llm=llm, prompt=prompt_2, output_key="keywords_list")
se_chain = SequentialChain(chains=[chain3, chain4], input_variables=["keywords"],
                           output_variables=["text_content", "keywords_list"], verbose=True)


NameError: name 'output_parser' is not defined

In [29]:
result_se_chain = se_chain({"keywords": "12岁的npc少年"})



> Entering new SequentialChain chain...

> Finished chain.


In [30]:
output_parser.parse(result_se_chain.get("text_content"))

['作为一个游戏原画设计师，我将以12岁的少年NPC位主角，为他设计出一个充满活力的形象，他的外表看上去精力旺盛，深究脸色稚嫩，衣着也相对时尚，活泼的模样能够引起人们的关注。他从表情到神态均灵动有致，让人迅速融入到游戏中去。']

In [31]:
output_parser.parse(result_se_chain.get("keywords_list"))

['少年NPC', '12岁', '精力旺盛', '稚嫩', '时尚', '活泼', '表情', '神态', '灵动', '有致', '引起关注.']


### Part-4.4：Chain -> LLMRouterChain、MultiPromptChain：实现条件判断的大模型调用

In [48]:
from langchain import ConversationChain
from langchain.chains.router.multi_prompt_prompt import MULTI_PROMPT_ROUTER_TEMPLATE

templates = [
    {
        "name": "游戏设计师（Game Designer）",
        "description": "适合回答设计游戏的玩法、关卡、角色和系统，确保游戏的乐趣和挑战性。",
        "template": "你是游戏设计师，你需要根据下面的主题：{input}，给出10个关键词描述。"
    }, {
        "name": "角色设计师（Character Designer）",
        "description": "适合回答创造游戏中的各种角色，包括主角、配角、敌人、NPC（非玩家角色）等",
        "template": "你是角色设计师，你需要根据下面的主题：{input}，给出10个关键词描述。"
    }, {
        "name": "关卡设计师（Level Designer）",
        "description": "适合回答设计游戏中的各个关卡、场景和环境",
        "template": "你是关卡设计师，你需要根据下面的主题：{input}，给出10个关键词描述。"
    }, {
        "name": "美术设计师（Art Designer）",
        "description": "适合回答游戏的视觉风格、角色造型、场景设计等美术方面的工作。",
        "template": "你是美术设计师，你需要根据下面的主题：{input}，给出10个关键词描述。"
    }
]

destination_chains = {}
for tem in templates:
    name = tem['name']
    description = tem["description"]
    template = tem["template"]
    prompt = PromptTemplate(template=template, input_variables=["input"])
    chain = LLMChain(llm=llm, prompt=prompt, verbose=True)
    destination_chains[name] = chain

default_chain = ConversationChain(llm=llm, output_key="text")


In [49]:
from langchain.chains.router.llm_router import RouterOutputParser, LLMRouterChain

destinations = [f"{p['name']}: {p['description']}" for p in templates]
destinations_str = "\n".join(destinations)
router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(destinations=destinations_str)
print(router_template)

router_prompt = PromptTemplate(
    template=router_template,
    input_variables=["input"],
    output_parser=RouterOutputParser()

)

router_chain = LLMRouterChain.from_llm(llm=llm, prompt=router_prompt, verbose=True)


Given a raw text input to a language model select the model prompt best suited for the input. You will be given the names of the available prompts and a description of what the prompt is best suited for. You may also revise the original input if you think that revising it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ a potentially modified version of the original input
}}
```

REMEMBER: "destination" MUST be one of the candidate prompt names specified below OR it can be "DEFAULT" if the input is not well suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
游戏设计师（Game Designer）: 适合回答设计游戏的玩法、关卡、角色和系统，确保游戏的乐趣和挑战性。
角色设计师（Character Designer）: 适合回答创造游戏中的各种角色，包括主

In [50]:
from langchain.chains import MultiPromptChain

mult_chain = MultiPromptChain(
    router_chain=router_chain,
    default_chain=default_chain,
    destination_chains=destination_chains,
    verbose=True
)

In [43]:
print(mult_chain.run({"input": "设计一个6岁的ncp小男孩的特征"}))



> Entering new MultiPromptChain chain...


> Entering new LLMRouterChain chain...

> Finished chain.
角色设计师（Character Designer）: {'input': '设计一个6岁的 NPC 小男孩的外貌特征'}

> Entering new LLMChain chain...
Prompt after formatting:
你是角色设计师，你需要根据下面的主题：设计一个6岁的 NPC 小男孩的外貌特征，给出10个关键词描述。

> Finished chain.

> Finished chain.


1. 甜美  2. 聪明    3. 剪短头发  4. 胖乎乎的肚子  5. 绿色眼睛 6. 可爱的笑容 7. 小巧的身躯 8. 用蓝色衣衫搭配紫色裤子 9. 喜欢玩滑板 10. 爱好音乐


In [53]:
result = mult_chain.run({'input':'设计一个宇宙空间站的游戏场景'})



> Entering new MultiPromptChain chain...


> Entering new LLMRouterChain chain...

> Finished chain.
关卡设计师（Level Designer）: {'input': '设计一个宇宙空间站游戏的关卡、场景和环境'}

ValueError: Missing some input keys: {'subject'}

## Part-5：Memory


## Part-6：Agent
